<a href="https://colab.research.google.com/github/charoo-rumsan/community_tool_R-D/blob/main/task2_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install polars

In [ ]:
!pip install scikit-learn

In [ ]:
import polars as pl

df = pl.read_csv("/content/first_100_rows (1) - first_100_rows (1).csv.csv")

# Display first few rows and schema
print("DataFrame Head:")
print(df.head())
print("\nSchema:")
print(df.schema)


DataFrame Head:
shape: (5, 353)
┌─────┬─────────────┬─────────────┬────────────┬───┬────────┬────────────┬────────────┬────────────┐
│     ┆ start       ┆ end         ┆ today      ┆ … ┆ घरबाट  ┆ (घरबाट     ┆ (नजिकैको    ┆ (Distance  │
│ --- ┆ ---         ┆ ---         ┆ ---        ┆   ┆ स्वास्थ्य ┆ बजारको दुरी ┆ शौचालय र   ┆ to safe    │
│ i64 ┆ str         ┆ str         ┆ str        ┆   ┆ संस्थाको ┆ in METER   ┆ घरको दुरी   ┆ Shelter in │
│     ┆             ┆             ┆            ┆   ┆ दुरी …  ┆ ---        ┆ in…        ┆ M…         │
│     ┆             ┆             ┆            ┆   ┆ ---    ┆ i64        ┆ ---        ┆ ---        │
│     ┆             ┆             ┆            ┆   ┆ i64    ┆            ┆ str        ┆ i64        │
╞═════╪═════════════╪═════════════╪════════════╪═══╪════════╪════════════╪════════════╪════════════╡
│ 0   ┆ 2022-04-13T ┆ 2022-04-13T ┆ 2022-04-13 ┆ … ┆ 2      ┆ 100        ┆ null       ┆ 100        │
│     ┆ 10:14:57.94 ┆ 10:49:51.21 ┆            ┆  

Story 2:

The ML Field Classifier
As a HR manager
 I want to upload employee data from different sources
 So that the system automatically identifies email fields, phone numbers, and addresses

How it works:
System sees: "john.doe@company.com" in column “Contact Info”
ML model recognizes email pattern → suggests mapping to “email_address”
System sees: "(555) 123-4567" in column “Phone”
ML model recognizes phone pattern → suggests mapping to “phone_number”
No manual rules needed - the AI figures it out!


In [ ]:
import polars as pl
import re

# Load the dataset (use your full CSV path or the sample)
df = pl.read_csv("/content/first_100_rows (1) - first_100_rows (1).csv.csv")

def classify_field(column_name, sample_values):
    """
    Classify a column into types: phone, email, name, address, or other.
    Args:
        column_name (str): Column header (e.g., "phonenumber").
        sample_values (list): Sample data from the column.
    Returns:
        str: Field type.
    """
    column_name = column_name.lower()

    # Phone: Contains "phone" or matches 10-digit pattern
    if "phone" in column_name or any(re.match(r'^\d{10}$', str(val)) for val in sample_values if val):
        return "phone"

    # Email: Contains "email" or matches email pattern
    if "email" in column_name or any(re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', str(val)) for val in sample_values if val):
        return "email"

    # Name: Contains "name" or text with spaces/capitalized words
    if "name" in column_name or any(isinstance(val, str) and " " in val and val[0].isupper() for val in sample_values if val):
        return "name"

    # Address: Contains "municipality", "ward", "tole", "house"
    if any(k in column_name for k in ["municipality", "ward", "tole", "house"]):
        return "address"

    return "other"

# Classify all columns
field_classifications = {}
for col in df.columns:
    sample_values = df[col].head(5).to_list()  # Sample first 5 values
    field_classifications[col] = classify_field(col, sample_values)

# Display classifications
print("Field Classifications:")
for col, field_type in field_classifications.items():
    print(f"{col}: {field_type}")

# Example output to a file for review
with open("/content/field_classifications.txt", "w") as f:
    for col, field_type in field_classifications.items():
        f.write(f"{col}: {field_type}\n")

Field Classifications:
: other
start: other
end: other
today: other
username: name
simserial: other
subscriberid: other
deviceid: other
phonenumber: phone
General Questions/Municipality and Ward Details/Name of Municipality (नगरपालिकाको नाम): name
General Questions/Municipality and Ward Details/Ward Number (वडा नं .): name
General Questions/Municipality and Ward Details/Ward Number (वडा नं ): name
General Questions/Name of the Tole (सर्वेक्षण भैरहेको स्थानको नाम): name
General Questions/House No. (घर नं): address
GPS Coordinates: other
General Questions/_GPS Coordinates_latitude: other
General Questions/_GPS Coordinates_longitude: other
General Questions/_GPS Coordinates_altitude: other
General Questions/_GPS Coordinates_precision: other
General Questions/Do I have your consent to conduct this survey? (के यो सर्भे गर्नमा तपाइको सहमति छ?): name
Basic Details of House Owner/Information on house owner/Name of the house owner (घरमुलीको नाम): name
Basic Details of House Owner/Information on

Story 3: The Smart Pattern Recognizer

As a finance team member
 I want to upload expense reports that sometimes have “Amount”, “Total”, “Cost”, or “Price”
 So that the system learns these all mean the same thing - money fields

How it works:
First upload: You manually tell system “Amount” = “expense_amount”
Second upload: System sees “Total” and suggests “This looks like a money field, map to expense_amount?”
Third upload: System automatically maps “Cost” and “Price” with 90% confidence
System learns: Money-related words → financial fields


In [ ]:
import polars as pl
import difflib

# Load the dataset
df = pl.read_csv("/content/first_100_rows (1) - first_100_rows (1).csv.csv")

# Synonym dictionary for categories
synonym_dict = {
    "personal": ["name", "age", "gender", "marital"],
    "address": ["municipality", "ward", "tole", "house", "gps"],
    "financial": ["income", "expense", "amount"],
    "demographic": ["family", "members", "female", "male"],
    "identity": ["citizenship", "id", "document"],
    "contact": ["phone", "email"]
}

def find_category(column_name):
    """
    Map column to a category based on synonyms and similarity.
    Args:
        column_name (str): Column header.
    Returns:
        str: Category name.
    """
    column_name = column_name.lower()
    best_score = 0
    best_category = "other"

    for category, synonyms in synonym_dict.items():
        for synonym in synonyms:
            # Similarity score using difflib
            score = difflib.SequenceMatcher(None, column_name, synonym).ratio()
            if score > best_score and score > 0.5:  # Threshold for match
                best_score = score
                best_category = category
            # Exact keyword match
            if synonym in column_name:
                return category

    return best_category

# Map columns to categories
category_mappings = {col: find_category(col) for col in df.columns}

# Display mappings
print("Category Mappings:")
for col, category in category_mappings.items():
    print(f"{col}: {category}")

# Save to file
with open("/content/category_mappings.txt", "w") as f:
    for col, category in category_mappings.items():
        f.write(f"{col}: {category}\n")

Category Mappings:
: other
start: other
end: personal
today: other
username: personal
simserial: personal
subscriberid: identity
deviceid: identity
phonenumber: contact
General Questions/Municipality and Ward Details/Name of Municipality (नगरपालिकाको नाम): personal
General Questions/Municipality and Ward Details/Ward Number (वडा नं .): address
General Questions/Municipality and Ward Details/Ward Number (वडा नं ): address
General Questions/Name of the Tole (सर्वेक्षण भैरहेको स्थानको नाम): personal
General Questions/House No. (घर नं): address
GPS Coordinates: address
General Questions/_GPS Coordinates_latitude: address
General Questions/_GPS Coordinates_longitude: address
General Questions/_GPS Coordinates_altitude: address
General Questions/_GPS Coordinates_precision: address
General Questions/Do I have your consent to conduct this survey? (के यो सर्भे गर्नमा तपाइको सहमति छ?): other
Basic Details of House Owner/Information on house owner/Name of the house owner (घरमुलीको नाम): personal


Story 4: The Confidence Scorer

As a operations manager
 I want to see how confident the system is about each mapping
 So that I can review uncertain mappings before processing

How it works:
System shows you:
 “First Name” → “first_name” (95% confident)
 “Email” → “email_address” (92% confident)
 “Info” → “description” (65% confident) - Please review
 “XYZ123” → ??? (15% confident) - Needs manual mapping


In [ ]:
import polars as pl
import difflib

# Load the dataset
df = pl.read_csv("/content/first_100_rows (1) - first_100_rows (1).csv.csv")

# Synonym dictionary (same as Story 3)
synonym_dict = {
    "personal": ["name", "age", "gender", "marital"],
    "address": ["municipality", "ward", "tole", "house", "gps"],
    "financial": ["income", "expense", "amount"],
    "demographic": ["family", "members", "female", "male"],
    "identity": ["citizenship", "id", "document"],
    "contact": ["phone", "email"]
}

def score_mapping(column_name):
    """
    Map column to category with confidence score.
    Args:
        column_name (str): Column header.
    Returns:
        tuple: (category, confidence score).
    """
    column_name = column_name.lower()
    best_score = 0
    best_category = "other"

    for category, synonyms in synonym_dict.items():
        for synonym in synonyms:
            # Exact match: high confidence
            if synonym in column_name:
                return (category, 95)
            # Fuzzy match
            score = difflib.SequenceMatcher(None, column_name, synonym).ratio() * 100
            if score > best_score and score > 50:
                best_score = score
                best_category = category

    return (best_category, best_score if best_score > 0 else 50)

# Map columns with confidence scores
confidence_mappings = {col: score_mapping(col) for col in df.columns}

# Display mappings
print("Confidence Mappings:")
for col, (category, score) in confidence_mappings.items():
    print(f"{col}: {category} (Confidence: {score:.1f}%)")

# Save to file
with open("/content/confidence_mappings.txt", "w") as f:
    for col, (category, score) in confidence_mappings.items():
        f.write(f"{col}: {category} (Confidence: {score:.1f}%)\n")

Confidence Mappings:
: other (Confidence: 50.0%)
start: other (Confidence: 50.0%)
end: personal (Confidence: 66.7%)
today: other (Confidence: 50.0%)
username: personal (Confidence: 95.0%)
simserial: personal (Confidence: 62.5%)
subscriberid: identity (Confidence: 95.0%)
deviceid: identity (Confidence: 95.0%)
phonenumber: contact (Confidence: 95.0%)
General Questions/Municipality and Ward Details/Name of Municipality (नगरपालिकाको नाम): personal (Confidence: 95.0%)
General Questions/Municipality and Ward Details/Ward Number (वडा नं .): address (Confidence: 95.0%)
General Questions/Municipality and Ward Details/Ward Number (वडा नं ): address (Confidence: 95.0%)
General Questions/Name of the Tole (सर्वेक्षण भैरहेको स्थानको नाम): personal (Confidence: 95.0%)
General Questions/House No. (घर नं): address (Confidence: 95.0%)
GPS Coordinates: address (Confidence: 95.0%)
General Questions/_GPS Coordinates_latitude: address (Confidence: 95.0%)
General Questions/_GPS Coordinates_longitude: address

Story 5: Creating Mapping Rules

As a system administrator
 I want to create standard mapping rules for my organization
 So that everyone’s spreadsheets get mapped consistently
Configuration looks like:

yaml

standard_mappings:
  customer_fields:
    - "Customer Name" → "customer_name"
    - "Client Name" → "customer_name"
    - "Company" → "customer_name"
  
  financial_fields:
    - "Amount|Total|Cost|Price" → "amount"
    - "Date|Transaction Date" → "transaction_date"


In [ ]:
!pip install pyyaml

In [ ]:
import polars as pl
import yaml

# Load the dataset
df = pl.read_csv("/content/first_100_rows (1) - first_100_rows (1).csv.csv")

# Define standard schema (YAML)
schema = """
personal:
  name: string
  age: integer
  gender: string
  marital_status: string
address:
  municipality: string
  ward: string
  tole: string
  house_no: string
contact:
  phone: string
identity:
  citizenship: string
demographic:
  total_members: integer
  total_female: integer
  total_male: integer
"""
standard_schema = yaml.safe_load(schema)

# Mapping rules based on Story 3 logic
mapped_standard_names = set()

def map_to_standard(column_name):
    column_name = column_name.lower()
    standard_name = None

    if "name of the house owner" in column_name:
        standard_name = "personal.name"
    elif "age of the house owner" in column_name:
        standard_name = "personal.age"
    elif "gender of house owner" in column_name:
        standard_name = "personal.gender"
    elif "marital status" in column_name:
        standard_name = "personal.marital_status"
    # Prioritize "ward" over "municipality"
    elif "ward" in column_name:
        standard_name = "address.ward"
    elif "municipality" in column_name:
        standard_name = "address.municipality"
    elif "tole" in column_name:
        standard_name = "address.tole"
    elif "house no" in column_name:
        standard_name = "address.house_no"
    elif "phone" in column_name:
        standard_name = "contact.phone"
    elif "citizenship" in column_name:
        standard_name = "identity.citizenship"
    elif "total_members" in column_name:
        standard_name = "demographic.total_members"
    elif "total_female" in column_name:
        standard_name = "demographic.total_female"
    elif "total_male" in column_name:
        standard_name = "demographic.total_male"

    if standard_name and standard_name not in mapped_standard_names:
        mapped_standard_names.add(standard_name)
        return standard_name
    else:
        return None


# Apply mappings
standard_columns = {}
for col in df.columns:
    standard_col = map_to_standard(col)
    if standard_col:
        standard_columns[col] = standard_col

# Create standardized DataFrame
standard_df = df.select(list(standard_columns.keys())).rename(standard_columns)

# Display and save
print("Standardized DataFrame:")
print(standard_df.head())
standard_df.write_csv("/content/standardized_data.csv")

Standardized DataFrame:
shape: (5, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ contact.p ┆ address.w ┆ address.t ┆ address.h ┆ … ┆ demograph ┆ demograph ┆ address.m ┆ identity │
│ hone      ┆ ard       ┆ ole       ┆ ouse_no   ┆   ┆ ic.total_ ┆ ic.total_ ┆ unicipali ┆ .citizen │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ male      ┆ members   ┆ ty        ┆ ship     │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ i64       ┆ i64       ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ null      ┆ Laljhadi  ┆ Danda     ┆ NDHS/88   ┆ … ┆ 2         ┆ 4         ┆ null      ┆ 353/044  │
│           ┆ Rural Mun ┆ Bichwa    ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ icipality ┆           ┆           ┆   

Story 6: Field Schema Templates

As a data manager
 I want to define what my final JSON should look like
 So that all mapped data follows the same structure
Template you create:

json
{
  "customer_info": {
    "name": "string",
    "email": "email_format",
    "phone": "phone_format"
  },
  "transaction": {
    "amount": "decimal",
    "date": "date_format",
    "description": "string"
  }
}

What happens:
System ensures all mappings fit this structure
Validates data types (email must be valid email, etc.)
Rejects mappings that don’t match your schema


In [ ]:
!pip install polars
!pip install pyyaml

import polars as pl
import yaml

# Load the standardized DataFrame
standard_df = pl.read_csv("/content/standardized_data.csv")

# Load schema
schema = """
personal:
  name: string
  age: integer
  gender: string
  marital_status: string
address:
  municipality: string
  ward: string
  tole: string
  house_no: string
contact:
  phone: string
identity:
  citizenship: string
demographic:
  total_members: integer
  total_female: integer
  total_male: integer
"""
schema_dict = yaml.safe_load(schema)

def validate_schema(df, schema):
    """
    Validate DataFrame against schema.
    Args:
        df: Polars DataFrame.
        schema: Schema dictionary.
    Returns:
        list: Validation errors.
    """
    errors = []
    schema_fields = {}
    for category, fields in schema.items():
        for field, dtype in fields.items():
            schema_fields[f"{category}.{field}"] = dtype

    for col in df.columns:
        if col not in schema_fields:
            errors.append(f"Column {col} not in schema")
            continue
        expected_dtype = schema_fields[col]
        actual_dtype = str(df[col].dtype)
        if (expected_dtype == "string" and "String" not in actual_dtype) or \
           (expected_dtype == "integer" and "Int" not in actual_dtype):
            errors.append(f"Column {col}: Expected {expected_dtype}, got {actual_dtype}")

    return errors

# Validate
errors = validate_schema(standard_df, schema_dict)
print("Validation Errors:")
if errors:
    for error in errors:
        print(error)
else:
    print("No errors found")

# Save errors to file
with open("/content/validation_errors.txt", "w") as f:
    if errors:
        for error in errors:
            f.write(f"{error}\n")
    else:
        f.write("No errors found\n")

Validation Errors:
No errors found


Story 7: User Preferences & Learning

As a frequent user
 I want to the system to remember my preferences
 So that I don’t have to re-map the same fields every time

Your preferences saved:
“I always want ‘Date’ fields to map to ‘created_date’”
“I prefer snake_case naming (customer_name, not customerName)”
“Always ask me before mapping fields with less than 80% confidence”

System remembers:
Your past mapping decisions
Fields you commonly use
Your naming conventions


In [ ]:
!pip install polars

import polars as pl
import json

# Load the dataset
df = pl.read_csv("/content/first_100_rows (1) - first_100_rows (1).csv.csv")

# Load or define mappings (from Story 5 logic)
def map_to_standard(column_name):
    column_name = column_name.lower()
    if "name of the house owner" in column_name:
        return "personal.name"
    elif "age of the house owner" in column_name:
        return "personal.age"
    elif "gender of house owner" in column_name:
        return "personal.gender"
    # Add other mappings as in Story 5
    return None

# Create mappings
mappings = {col: map_to_standard(col) for col in df.columns if map_to_standard(col)}

# Save mappings to JSON
with open("/content/mappings.json", "w") as f:
    json.dump(mappings, f, indent=2)

# Apply saved mappings to a new dataset (simulated here with same dataset)
with open("/content/mappings.json", "r") as f:
    saved_mappings = json.load(f)

new_df = df.select(list(saved_mappings.keys())).rename(saved_mappings)
print("DataFrame with Saved Mappings:")
print(new_df.head())

# Save output
new_df.write_csv("/content/data_with_saved_mappings.csv")

DataFrame with Saved Mappings:
shape: (5, 3)
┌───────────────────────┬──────────────┬─────────────────┐
│ personal.name         ┆ personal.age ┆ personal.gender │
│ ---                   ┆ ---          ┆ ---             │
│ str                   ┆ i64          ┆ str             │
╞═══════════════════════╪══════════════╪═════════════════╡
│ Kedar Rana            ┆ 42           ┆ Male (पुरुष)      │
│ Lemba Tamang          ┆ 72           ┆ Male (पुरुष)      │
│ Bhakta Bahadur Tamang ┆ 33           ┆ Male (पुरुष)      │
│ Mauri chand           ┆ 40           ┆ Male (पुरुष)      │
│ Bir sing saud         ┆ 37           ┆ Male (पुरुष)      │
└───────────────────────┴──────────────┴─────────────────┘


Story 8: Dynamic Rule Updates

As a team lead
 I want to update mapping rules based on what we learn
 So that the system gets smarter over time

Scenario:

Week 1: System maps “Client ID” → “customer_id” (manual)

Week 2: System maps “Cust ID” → “customer_id” (learned)

Week 3: You add rule: “Any field with ‘ID’ + customer/client/cust → customer_id”

Week 4: System automatically maps “Customer_ID”, “CLIENT-ID”, “cust_identifier”


In [ ]:
!pip install polars

import polars as pl
import json

# Load the dataset
df = pl.read_csv("/content/first_100_rows (1) - first_100_rows (1).csv.csv")

# Initial synonym dictionary
synonym_dict = {
    "personal": ["name", "age", "gender", "marital"],
    "address": ["municipality", "ward", "tole", "house"],
    "contact": ["phone"],
    "identity": ["citizenship"],
    "demographic": ["family", "members", "female", "male"]
}

# Simulate learning: Add Nepali terms
def learn_mappings(df, synonym_dict):
    for col in df.columns:
        if "छ" in col:  # Detect Nepali "Yes"
            synonym_dict["boolean"] = synonym_dict.get("boolean", []) + ["छ"]
        # Add more learning rules (e.g., detect "नं" for number/ID)
        if "नं" in col:
            synonym_dict["identity"] = synonym_dict["identity"] + ["नं"]
    return synonym_dict

# Update synonyms
updated_synonyms = learn_mappings(df, synonym_dict)

# Save updated synonyms
with open("/content/updated_synonyms.json", "w") as f:
    json.dump(updated_synonyms, f, indent=2)

# Apply updated synonyms to map columns
def map_with_updated_synonyms(column_name, synonym_dict):
    column_name = column_name.lower()
    for category, synonyms in synonym_dict.items():
        if any(syn in column_name for syn in synonyms):
            return category
    return "other"

mappings = {col: map_with_updated_synonyms(col, updated_synonyms) for col in df.columns}
print("Updated Mappings:")
for col, category in mappings.items():
    print(f"{col}: {category}")

# Save mappings
with open("/content/updated_mappings.txt", "w") as f:
    for col, category in mappings.items():
        f.write(f"{col}: {category}\n")

Updated Mappings:
: other
start: other
end: other
today: other
username: personal
simserial: other
subscriberid: other
deviceid: other
phonenumber: contact
General Questions/Municipality and Ward Details/Name of Municipality (नगरपालिकाको नाम): personal
General Questions/Municipality and Ward Details/Ward Number (वडा नं .): address
General Questions/Municipality and Ward Details/Ward Number (वडा नं ): address
General Questions/Name of the Tole (सर्वेक्षण भैरहेको स्थानको नाम): personal
General Questions/House No. (घर नं): address
GPS Coordinates: other
General Questions/_GPS Coordinates_latitude: other
General Questions/_GPS Coordinates_longitude: other
General Questions/_GPS Coordinates_altitude: other
General Questions/_GPS Coordinates_precision: other
General Questions/Do I have your consent to conduct this survey? (के यो सर्भे गर्नमा तपाइको सहमति छ?): boolean
Basic Details of House Owner/Information on house owner/Name of the house owner (घरमुलीको नाम): personal
Basic Details of Hous